In [1]:
!pip install "pymilvus[model]" openai requests tqdm torch

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import os
api_key = os.getenv('DEEPSEEK_API_KEY')

In [3]:
from glob import glob
filepath = '././milvus_docs/en/faq/*.md'
text_lines = []

for i in glob(filepath, recursive=True):
    with open(i,'r') as f:
        file_text =f.read()
        
    text_lines += file_text.split('# ')
#print(len(text_lines))
print(len(text_lines))

72


In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",
)

In [5]:
#定义一个 embedding 模型，使用 milvus_model 来生成文本嵌入。我们以 DefaultEmbeddingFunction 模型为例，这是一个预训练的轻量级嵌入模型。

In [6]:
from pymilvus import model as milvus_model
embedding_model = milvus_model.DefaultEmbeddingFunction()
embedding_model

/root/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#测试数据，生成embedding,打印纬度数和前几个纬度的数值

In [8]:
test_embedding = embedding_model.encode_queries(["This is a hahah"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[ 0.01634293  0.03422707 -0.00944707 -0.058086    0.01152717 -0.00139032
  0.0201766  -0.00700357 -0.01878644 -0.01590427]


In [9]:
#创建向量数据库，创建 Collection，将数据加载到 Milvus,

In [10]:
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="./milvus_demo.db")
collection_name = "my_rag_collection"

#如果库已存在就删除
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [11]:
#创建一个具有指定参数的新 collection。
#如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 id 字段作为主键，以及一个 vector 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

In [12]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。
)

In [13]:
#插入数据

In [14]:
from tqdm import tqdm
data = []
doc_embeddings = embedding_model.encode_documents(text_lines)
doc_embeddings
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
    
milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 72/72 [00:00<00:00, 384211.05it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}

In [15]:
#检索查询数据

In [16]:
question = "why pods is crashed?"
search_res = milvus_client.search(
    collection_name=collection_name,
    data = embedding_model.encode_queries([question]),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [17]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "3. Multiple pods keep crashing while `etcd-0` is still running\n\n  You can run the following code if multiple pods keeps crashing while `etcd-0` is still running.\n  \n  ```\n  kubectl scale sts <etcd-sts> --replicas=1\n  ",
        0.5926371216773987
    ],
    [
        "4. All pods crash\n  \n  When all pods crash, try copying the `/bitnami/etcd/data/member/snap/db` file. Use `https://github.com/etcd-io/bbolt` to modify database data.\n\n  All Milvus metadata are kept in the `key` bucket. Back up the data in this bucket and run the following commands. Note that the prefix data in the `by-dev/meta/session` file does not require a backup.\n  \n  ```\n  kubectl kubectl scale sts <etcd-sts> --replicas=0\n  ",
        0.579849362373352
    ],
    [
        "2. etcd pod crash\n\n  When an etcd pod crashes with `Error: bad member ID arg (strconv.ParseUint: parsing \"\": invalid syntax), expecting ID in Hex`, you can log into this pod and delete the `/bitnami/etcd/data/mem

In [18]:
#使用 LLM 获取 RAG 响应
#将检索到的文档转换为字符串格式。

In [19]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
context

'3. Multiple pods keep crashing while `etcd-0` is still running\n\n  You can run the following code if multiple pods keeps crashing while `etcd-0` is still running.\n  \n  ```\n  kubectl scale sts <etcd-sts> --replicas=1\n  \n4. All pods crash\n  \n  When all pods crash, try copying the `/bitnami/etcd/data/member/snap/db` file. Use `https://github.com/etcd-io/bbolt` to modify database data.\n\n  All Milvus metadata are kept in the `key` bucket. Back up the data in this bucket and run the following commands. Note that the prefix data in the `by-dev/meta/session` file does not require a backup.\n  \n  ```\n  kubectl kubectl scale sts <etcd-sts> --replicas=0\n  \n2. etcd pod crash\n\n  When an etcd pod crashes with `Error: bad member ID arg (strconv.ParseUint: parsing "": invalid syntax), expecting ID in Hex`, you can log into this pod and delete the `/bitnami/etcd/data/member_id` file.\n\n  ##'

In [21]:
SYSTEM_PROMPT = """
Human: 请你从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。请在头部追加问题，最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。请在头部追加问题，最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n3. Multiple pods keep crashing while `etcd-0` is still running\n\n  You can run the following code if multiple pods keeps crashing while `etcd-0` is still running.\n  \n  ```\n  kubectl scale sts <etcd-sts> --replicas=1\n  \n4. All pods crash\n  \n  When all pods crash, try copying the `/bitnami/etcd/data/member/snap/db` file. Use `https://github.com/etcd-io/bbolt` to modify database data.\n\n  All Milvus metadata are kept in the `key` bucket. Back up the data in this bucket and run the following commands. Note that the prefix data in the `by-dev/meta/session` file does not require a backup.\n  \n  ```\n  kubectl kubectl scale sts <etcd-sts> --replicas=0\n  \n2. etcd pod crash\n\n  When an etcd pod crashes with `Error: bad member ID arg (strconv.ParseUint: parsing "": invalid syntax), expecting ID in Hex`, you can log into this pod and delete the `/bitnami/etcd/

In [22]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

why pods is crashed?
根据上下文，pods崩溃的原因包括：1. 当etcd-0仍在运行时多个pods持续崩溃；2. 所有pods崩溃；3. 当etcd pod因"Error: bad member ID arg (strconv.ParseUint: parsing "": invalid syntax), expecting ID in Hex"错误而崩溃。

<translated>
为什么pods崩溃了？
根据上下文，pods崩溃的原因包括：1. 当etcd-0仍在运行时多个pods持续崩溃；2. 所有pods崩溃；3. 当etcd pod因"Error: bad member ID arg (strconv.ParseUint: parsing "": invalid syntax), expecting ID in Hex"错误而崩溃。
</translated>
